In [1]:
from bs4 import BeautifulSoup
import pandas as pd
import urllib3

Waiting for a Spark session to start...
Spark Initialization Done! ApplicationId = app-20181015133006-0000


#### Scraping wiki web page

In [2]:
http = urllib3.PoolManager()
url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
response = http.request('GET', url)
soup = BeautifulSoup(response.data)
data = []
colname=[]
table=soup.find('table', attrs={'class': 'wikitable sortable'})
table
table_body = table.find('tbody')
table_body
colsname=table_body.find_all('th')
colsname=[ele.text.strip() for ele in colsname]
rows = table_body.find_all('tr')
for row in rows:
    cols = row.find_all('td')
    cols = [ele.text.strip() for ele in cols]
    data.append([ele for ele in cols if ele])

/opt/ibm/conda/miniconda3/lib/python3.5/site-packages/urllib3/connectionpool.py:858: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)
/opt/ibm/conda/miniconda3/lib/python3.5/site-packages/bs4/__init__.py:181: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 193 of the file /opt/ibm/conda/miniconda3/lib/python3.5/runpy.py. To get rid of this warning, change code that looks like this:

 BeautifulSoup(YOUR_MARKUP})

to this:

 BeautifulSoup(YOUR_MARKUP, "lxml")

  markup_type=markup_type))


#### creating dataframe

In [29]:
df = pd.DataFrame.from_records(data, columns=colsname)

#### filtering all conditions 


In [30]:
df=df.drop([0,0],axis=0)
df=df[(df.Borough != 'Not assigned') | (df.Neighbourhood !='Not assigned')]
df=df.reset_index(drop=True)
df=df.groupby(['Postcode','Borough'])['Neighbourhood'].apply(','.join).reset_index()
df.loc[df['Neighbourhood'] == 'Not assigned', 'Neighbourhood'] = df['Borough']
df.shape

(103, 3)